In [1]:
from modularity_aware_gae.evaluation import community_detection, link_prediction
from modularity_aware_gae.input_data import load_data, load_labels
from modularity_aware_gae.louvain import louvain_clustering
from modularity_aware_gae.model import  GCNModelAE, GCNModelVAE, LinearModelAE, LinearModelVAE
from modularity_aware_gae.optimizer import OptimizerAE, OptimizerVAE
from modularity_aware_gae.preprocessing import *
from modularity_aware_gae.sampling import get_distribution, node_sampling
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Parameters related to the input data
flags.DEFINE_string('dataset', 'email', 'Graph dataset, among: cora, citeseer, wiki, celegans, email, polbooks, texas, wisconsin')

flags.DEFINE_boolean('features', False, 'Whether to include node features')


# Parameters related to the Modularity-Aware GAE/VGAE model to train

# 1/3 - General parameters associated with GAE/VGAE
flags.DEFINE_string('model', 'linear_vae', 'Model to train, among: gcn_ae, gcn_vae, \
                                            linear_ae, linear_vae')
flags.DEFINE_float('dropout', 0., 'Dropout rate')
flags.DEFINE_integer('iterations', 200, 'Number of iterations in training')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (Adam)')
flags.DEFINE_integer('hidden', 32, 'Dimension of the GCN hidden layer')
flags.DEFINE_integer('dimension', 16, 'Dimension of the output layer, i.e., \
                                       dimension of the embedding space')

# 2/3 - Additional parameters, specific to Modularity-Aware models
flags.DEFINE_float('beta', 0.0, 'Beta hyperparameter of Mod.-Aware models')
flags.DEFINE_float('lamb', 0.0, 'Lambda hyperparameter of Mod.-Aware models')
flags.DEFINE_float('gamma', 1.0, 'Gamma hyperparameter of Mod.-Aware models')
flags.DEFINE_integer('s_reg', 2, 's hyperparameter of Mod.-Aware models')

# 3/3 - Additional parameters, aiming to improve scalability
flags.DEFINE_boolean('fastgae', False, 'Whether to use the FastGAE framework')
flags.DEFINE_integer('nb_node_samples', 1000, 'In FastGAE, number of nodes to \
                                               sample at each iteration, i.e., \
                                               size of decoded subgraph')
flags.DEFINE_string('measure', 'degree', 'In FastGAE, node importance measure used \
                                          for sampling: degree, core, or uniform')
flags.DEFINE_float('alpha', 1.0, 'alpha hyperparameter associated with the FastGAE sampling')
flags.DEFINE_boolean('replace', False, 'Sample nodes with or without replacement')
flags.DEFINE_boolean('simple', True, 'Use simpler (and faster) modularity in optimizers')


# Parameters related to the experimental setup
flags.DEFINE_string('task', 'task_2', 'task_1: pure community detection \
                                       task_2: joint link prediction and \
                                               community detection')
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 1., 'Proportion of edges in validation set \
                                    for the link prediction task')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      for the link prediction task')
flags.DEFINE_boolean('validation', False, 'Whether to compute validation \
                                           results at each iteration, for \
                                           the link prediction task')
flags.DEFINE_boolean('verbose', True, 'Whether to print all comments details')


# Introductory message
if FLAGS.verbose:
    introductory_message()


 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: email
- Mode name: linear_vae
- Number of models to train: 10
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the output layer: 16
- lambda: 0.0
- beta: 0.0
- gamma: 1.0
- s: 2
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 



In [3]:
# Initialize lists to collect final results
mean_ami = []
mean_ari = []

mean_roc = []
mean_ap = []

# Load data
if FLAGS.verbose:
    print("LOADING DATA\n")
    print("Loading the", FLAGS.dataset, "graph")
adj_init, features_init = load_data(FLAGS.dataset)
#labels = load_labels(adj_init.shape[0])
if FLAGS.verbose:
    print("- Number of nodes:", adj_init.shape[0])
    #print("- Number of communities:", len(np.unique(labels)))
    print("- Use of node features:", FLAGS.features)
    print("Done! \n \n \n \n")

LOADING DATA

Loading the email graph
- Number of nodes: 986
- Use of node features: False
Done! 
 
 
 



In [4]:
# We repeat the entire training+test process FLAGS.nb_run times
for i in range(FLAGS.nb_run):
    if FLAGS.verbose:
        print("EXPERIMENTS ON MODEL", i + 1, "/", FLAGS.nb_run, "\n")
        print("STEP 1/3 - PREPROCESSING STEPS \n")


    # Edge masking for Link Prediction:
    if FLAGS.task == 'task_2':
        # Compute Train/Validation/Test sets
        if FLAGS.verbose:
            print("Masking some edges from the training graph, for link prediction")
            print("(validation set:", FLAGS.prop_val, "% of edges - test set:",
                  FLAGS.prop_test, "% of edges)")
        #adj, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)
        adj, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        adj = adj_init

    # Compute the number of nodes
    num_nodes = adj.shape[0]

    # Preprocessing on node features
    if FLAGS.verbose:
        print("Preprocessing node features")
    if FLAGS.features:
        features = features_init
    else:
        # If features are not used, replace feature matrix by identity matrix
        features = sp.identity(num_nodes)
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
    if FLAGS.verbose:
        print("Done! \n")


    # Community detection using Louvain, as a preprocessing step
    if FLAGS.verbose:
        print("Running the Louvain algorithm for community detection")
        print("as a preprocessing step for the encoder")
    # Get binary community matrix (adj_louvain_init[i,j] = 1 if nodes i and j are
    # in the same community) as well as number of communities found by Louvain
    adj_louvain_init, nb_communities_louvain, partition = louvain_clustering(adj, FLAGS.s_reg)
    if FLAGS.verbose:
        print("Done! Louvain has found", nb_communities_louvain, "communities \n")

    # FastGAE: node sampling for stochastic subgraph decoding - used when facing scalability issues
    if FLAGS.fastgae:
        if FLAGS.verbose:
            print("Preprocessing operations related to the FastGAE method")
        # Node-level p_i degree-based, core-based or uniform distribution
        node_distribution = get_distribution(FLAGS.measure, FLAGS.alpha, adj)
        # Node sampling for initializations
        sampled_nodes, adj_label, adj_sampled_sparse = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        sampled_nodes = np.array(range(FLAGS.nb_node_samples))


    # Placeholders
    if FLAGS.verbose:
        print('Setting up the model and the optimizer')
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_layer2': tf.sparse_placeholder(tf.float32), # Only used for 2-layer GCN encoders
        'degree_matrix': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ()),
        'sampled_nodes': tf.placeholder_with_default(sampled_nodes, shape = [FLAGS.nb_node_samples])
    }


    # Create model
    if FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    elif FLAGS.model == 'gcn_ae':
        # 2-layer GCN Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # 2-layer GCN Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    else:
        raise ValueError('Undefined model!')


    # Optimizer
    if FLAGS.fastgae:
        num_sampled = adj_sampled_sparse.shape[0]
        sum_sampled = adj_sampled_sparse.sum()
        pos_weight = float(num_sampled * num_sampled - sum_sampled) / sum_sampled
        norm = num_sampled * num_sampled / float((num_sampled * num_sampled - sum_sampled) * 2)
    else:
        pos_weight = float(num_nodes * num_nodes - adj.sum()) / adj.sum()
        norm = num_nodes * num_nodes / float((num_nodes * num_nodes - adj.sum()) * 2)

    if FLAGS.model in ('gcn_ae', 'linear_ae'):
        opt = OptimizerAE(preds = model.reconstructions,
                          labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                          degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                          num_nodes = num_nodes,
                          pos_weight = pos_weight,
                          norm = norm,
                          clusters_distance = model.clusters)
    else:
        opt = OptimizerVAE(preds = model.reconstructions,
                           labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                           degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                           model = model,
                           num_nodes = num_nodes,
                           pos_weight = pos_weight,
                           norm = norm,
                           clusters_distance = model.clusters)
    if FLAGS.verbose:
        print("Done! \n")


    # Symmetrically normalized "message passing" matrices
    if FLAGS.verbose:
        print("Preprocessing on message passing matrices")
    adj_norm = preprocess_graph(adj + FLAGS.lamb*adj_louvain_init)
    adj_norm_layer2 = preprocess_graph(adj)
    if not FLAGS.fastgae:
        adj_label = sparse_to_tuple(adj + sp.eye(num_nodes))
    if FLAGS.verbose:
        print("Done! \n")


    # Degree matrices
    deg_matrix, deg_matrix_init = preprocess_degree(adj, FLAGS.simple)


    # Initialize TF session
    if FLAGS.verbose:
        print("Initializing TF session")
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    if FLAGS.verbose:
        print("Done! \n")


    # Model training
    if FLAGS.verbose:
        print("STEP 2/3 - MODEL TRAINING \n")
        print("Starting training")

    for iter in range(FLAGS.iterations):

        # Flag to compute running time for each iteration
        t = time.time()

        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_norm_layer2, adj_label, features, deg_matrix, placeholders)
        if FLAGS.fastgae:
            # Update sampled subgraph and sampled Louvain matrix
            feed_dict.update({placeholders['sampled_nodes']: sampled_nodes})
            # New node sampling
            sampled_nodes, adj_label, adj_label_sparse, = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)

        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.cost_adj, opt.cost_mod],
                        feed_dict = feed_dict)

        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display information on the iteration
            print("Iteration:", '%04d' % (iter + 1), "Loss:", "{:.5f}".format(avg_cost),
                  "Time:", "{:.5f}".format(time.time() - t))

    if FLAGS.verbose:
        print("Done! \n")

    # Compute embedding vectors, for evaluation
    if FLAGS.verbose:
        print("STEP 3/3 - MODEL EVALUATION \n")
        print("Computing the final embedding vectors, for evaluation")
    emb = sess.run(model.z_mean, feed_dict = feed_dict)
    if FLAGS.verbose:
        print("Done! \n")

    # Test model: link prediction (classification edges/non-edges)

    if FLAGS.verbose:
        print("Testing: link prediction")
    # Get ROC and AP scores
    roc_score, ap_score = link_prediction(test_edges, test_edges_false, emb)
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)
    if FLAGS.verbose:
        print("Done! \n")

EXPERIMENTS ON MODEL 1 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 20 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.71865 Time: 0.25320
Iteration: 0002 Loss: 1.69200 Time: 0.04097
Iteration: 0003 Loss: 1.65148 Time: 0.04131
Iteration: 0004 Loss: 1.67516 Time: 0.04212
Iteration: 0005 Loss: 1.63269 Time: 0.04170
Iteration: 0006 Loss: 1.60129 Time: 0.04296
Iteration: 0007 Loss: 1.56575 Time: 0.04234
Iteration: 0008 Loss: 1.56472 Time: 0.04196
Iteration: 0009 Loss: 1.51188 Time: 0.04437
Iteration: 0010 Loss: 1.49677 Time: 0.04091
Iteration: 0011 Lo

Iteration: 0175 Loss: 0.63867 Time: 0.04119
Iteration: 0176 Loss: 0.63405 Time: 0.04119
Iteration: 0177 Loss: 0.63420 Time: 0.04057
Iteration: 0178 Loss: 0.63739 Time: 0.04200
Iteration: 0179 Loss: 0.63709 Time: 0.04056
Iteration: 0180 Loss: 0.62903 Time: 0.04200
Iteration: 0181 Loss: 0.63223 Time: 0.04026
Iteration: 0182 Loss: 0.63827 Time: 0.04203
Iteration: 0183 Loss: 0.62976 Time: 0.04240
Iteration: 0184 Loss: 0.62710 Time: 0.04354
Iteration: 0185 Loss: 0.62544 Time: 0.03961
Iteration: 0186 Loss: 0.62476 Time: 0.04084
Iteration: 0187 Loss: 0.62543 Time: 0.03901
Iteration: 0188 Loss: 0.63251 Time: 0.04194
Iteration: 0189 Loss: 0.62302 Time: 0.04008
Iteration: 0190 Loss: 0.62902 Time: 0.03993
Iteration: 0191 Loss: 0.62527 Time: 0.04200
Iteration: 0192 Loss: 0.62482 Time: 0.04009
Iteration: 0193 Loss: 0.63103 Time: 0.04113
Iteration: 0194 Loss: 0.62607 Time: 0.04194
Iteration: 0195 Loss: 0.62174 Time: 0.04092
Iteration: 0196 Loss: 0.62687 Time: 0.04135
Iteration: 0197 Loss: 0.61918 Ti

Iteration: 0151 Loss: 0.66087 Time: 0.04233
Iteration: 0152 Loss: 0.66176 Time: 0.03976
Iteration: 0153 Loss: 0.66346 Time: 0.04272
Iteration: 0154 Loss: 0.65712 Time: 0.04000
Iteration: 0155 Loss: 0.66658 Time: 0.04100
Iteration: 0156 Loss: 0.66317 Time: 0.04299
Iteration: 0157 Loss: 0.66273 Time: 0.04379
Iteration: 0158 Loss: 0.65622 Time: 0.04037
Iteration: 0159 Loss: 0.66253 Time: 0.04113
Iteration: 0160 Loss: 0.66008 Time: 0.04058
Iteration: 0161 Loss: 0.64794 Time: 0.04082
Iteration: 0162 Loss: 0.65864 Time: 0.04037
Iteration: 0163 Loss: 0.65722 Time: 0.04116
Iteration: 0164 Loss: 0.64814 Time: 0.03999
Iteration: 0165 Loss: 0.64733 Time: 0.04100
Iteration: 0166 Loss: 0.65259 Time: 0.04158
Iteration: 0167 Loss: 0.64971 Time: 0.04020
Iteration: 0168 Loss: 0.65077 Time: 0.04285
Iteration: 0169 Loss: 0.64785 Time: 0.04036
Iteration: 0170 Loss: 0.64419 Time: 0.04116
Iteration: 0171 Loss: 0.64206 Time: 0.03912
Iteration: 0172 Loss: 0.63978 Time: 0.03948
Iteration: 0173 Loss: 0.63985 Ti

Iteration: 0126 Loss: 0.69939 Time: 0.04191
Iteration: 0127 Loss: 0.69384 Time: 0.03903
Iteration: 0128 Loss: 0.69623 Time: 0.04094
Iteration: 0129 Loss: 0.69071 Time: 0.04100
Iteration: 0130 Loss: 0.69153 Time: 0.03901
Iteration: 0131 Loss: 0.69593 Time: 0.04047
Iteration: 0132 Loss: 0.69469 Time: 0.04052
Iteration: 0133 Loss: 0.68730 Time: 0.04004
Iteration: 0134 Loss: 0.68997 Time: 0.03815
Iteration: 0135 Loss: 0.68059 Time: 0.03875
Iteration: 0136 Loss: 0.68332 Time: 0.03914
Iteration: 0137 Loss: 0.68652 Time: 0.03984
Iteration: 0138 Loss: 0.67964 Time: 0.04002
Iteration: 0139 Loss: 0.68079 Time: 0.03879
Iteration: 0140 Loss: 0.67681 Time: 0.03974
Iteration: 0141 Loss: 0.68184 Time: 0.04100
Iteration: 0142 Loss: 0.67111 Time: 0.04247
Iteration: 0143 Loss: 0.66962 Time: 0.04096
Iteration: 0144 Loss: 0.66860 Time: 0.04071
Iteration: 0145 Loss: 0.66732 Time: 0.04126
Iteration: 0146 Loss: 0.66813 Time: 0.04091
Iteration: 0147 Loss: 0.67253 Time: 0.04099
Iteration: 0148 Loss: 0.66801 Ti

Iteration: 0101 Loss: 0.76130 Time: 0.04057
Iteration: 0102 Loss: 0.75593 Time: 0.04147
Iteration: 0103 Loss: 0.74466 Time: 0.04000
Iteration: 0104 Loss: 0.74814 Time: 0.04157
Iteration: 0105 Loss: 0.74935 Time: 0.04099
Iteration: 0106 Loss: 0.74529 Time: 0.04073
Iteration: 0107 Loss: 0.74660 Time: 0.04213
Iteration: 0108 Loss: 0.73704 Time: 0.04032
Iteration: 0109 Loss: 0.73271 Time: 0.04111
Iteration: 0110 Loss: 0.73071 Time: 0.04033
Iteration: 0111 Loss: 0.72177 Time: 0.04421
Iteration: 0112 Loss: 0.72651 Time: 0.04082
Iteration: 0113 Loss: 0.72229 Time: 0.03999
Iteration: 0114 Loss: 0.72936 Time: 0.03995
Iteration: 0115 Loss: 0.71903 Time: 0.04399
Iteration: 0116 Loss: 0.71712 Time: 0.04060
Iteration: 0117 Loss: 0.71188 Time: 0.04037
Iteration: 0118 Loss: 0.71733 Time: 0.04044
Iteration: 0119 Loss: 0.71171 Time: 0.04055
Iteration: 0120 Loss: 0.71541 Time: 0.03929
Iteration: 0121 Loss: 0.71135 Time: 0.04065
Iteration: 0122 Loss: 0.70774 Time: 0.04199
Iteration: 0123 Loss: 0.70966 Ti

...

Iteration: 0021 Loss: 1.32719 Time: 0.04195
Iteration: 0022 Loss: 1.29338 Time: 0.04200
Iteration: 0023 Loss: 1.29425 Time: 0.03958
Iteration: 0024 Loss: 1.25737 Time: 0.04341
Iteration: 0025 Loss: 1.24862 Time: 0.03876
Iteration: 0026 Loss: 1.24725 Time: 0.03912
Iteration: 0027 Loss: 1.22315 Time: 0.04110
Iteration: 0028 Loss: 1.19627 Time: 0.03876
Iteration: 0029 Loss: 1.18615 Time: 0.04073
Iteration: 0030 Loss: 1.19115 Time: 0.04100
Iteration: 0031 Loss: 1.16284 Time: 0.04104
Iteration: 0032 Loss: 1.16411 Time: 0.04224
Iteration: 0033 Loss: 1.15909 Time: 0.04175
Iteration: 0034 Loss: 1.12673 Time: 0.04097
Iteration: 0035 Loss: 1.11003 Time: 0.04300
Iteration: 0036 Loss: 1.11545 Time: 0.04004
Iteration: 0037 Loss: 1.08661 Time: 0.04299
Iteration: 0038 Loss: 1.09212 Time: 0.04198
Iteration: 0039 Loss: 1.07972 Time: 0.04295
Iteration: 0040 Loss: 1.07123 Time: 0.04023
Iteration: 0041 Loss: 1.03939 Time: 0.04101
Iteration: 0042 Loss: 1.04581 Time: 0.04100
Iteration: 0043 Loss: 1.04711 Ti

Done! Louvain has found 21 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.74930 Time: 0.28700
Iteration: 0002 Loss: 1.72900 Time: 0.04300
Iteration: 0003 Loss: 1.67894 Time: 0.04199
Iteration: 0004 Loss: 1.63794 Time: 0.04251
Iteration: 0005 Loss: 1.62056 Time: 0.04123
Iteration: 0006 Loss: 1.66034 Time: 0.04280
Iteration: 0007 Loss: 1.57189 Time: 0.04116
Iteration: 0008 Loss: 1.55823 Time: 0.04032
Iteration: 0009 Loss: 1.54182 Time: 0.04113
Iteration: 0010 Loss: 1.50015 Time: 0.04111
Iteration: 0011 Loss: 1.48068 Time: 0.04243
Iteration: 0012 Loss: 1.48182 Time: 0.04341
Iteration: 0013 Loss: 1.46157 Time: 0.04139
Iteration: 0014 Loss: 1.42600 Time: 0.04197
Iteration: 0015 Loss: 1.43370 Time: 0.04167
Iteration: 0016 Loss: 1.39320 Time: 0.04015
Iteration: 0017 Loss: 1.37437 Time: 0.04271
Iteration: 0018 Loss: 1.35488 Time: 

Iteration: 0183 Loss: 0.63006 Time: 0.04195
Iteration: 0184 Loss: 0.62828 Time: 0.04200
Iteration: 0185 Loss: 0.62358 Time: 0.04200
Iteration: 0186 Loss: 0.63213 Time: 0.04196
Iteration: 0187 Loss: 0.63031 Time: 0.04405
Iteration: 0188 Loss: 0.62862 Time: 0.04200
Iteration: 0189 Loss: 0.63071 Time: 0.04299
Iteration: 0190 Loss: 0.62598 Time: 0.04005
Iteration: 0191 Loss: 0.63090 Time: 0.04195
Iteration: 0192 Loss: 0.62625 Time: 0.03998
Iteration: 0193 Loss: 0.62078 Time: 0.04301
Iteration: 0194 Loss: 0.62556 Time: 0.04019
Iteration: 0195 Loss: 0.62597 Time: 0.04181
Iteration: 0196 Loss: 0.62170 Time: 0.04099
Iteration: 0197 Loss: 0.62101 Time: 0.04301
Iteration: 0198 Loss: 0.62070 Time: 0.04100
Iteration: 0199 Loss: 0.62226 Time: 0.04249
Iteration: 0200 Loss: 0.61425 Time: 0.04151
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 9 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some

Iteration: 0158 Loss: 0.65989 Time: 0.04153
Iteration: 0159 Loss: 0.65305 Time: 0.04194
Iteration: 0160 Loss: 0.65605 Time: 0.04143
Iteration: 0161 Loss: 0.65628 Time: 0.04236
Iteration: 0162 Loss: 0.64949 Time: 0.04099
Iteration: 0163 Loss: 0.64812 Time: 0.04115
Iteration: 0164 Loss: 0.64940 Time: 0.04099
Iteration: 0165 Loss: 0.64351 Time: 0.04100
Iteration: 0166 Loss: 0.64409 Time: 0.03997
Iteration: 0167 Loss: 0.64423 Time: 0.04158
Iteration: 0168 Loss: 0.64119 Time: 0.04086
Iteration: 0169 Loss: 0.64267 Time: 0.04198
Iteration: 0170 Loss: 0.63939 Time: 0.03980
Iteration: 0171 Loss: 0.64103 Time: 0.04106
Iteration: 0172 Loss: 0.63635 Time: 0.03928
Iteration: 0173 Loss: 0.63974 Time: 0.04010
Iteration: 0174 Loss: 0.64118 Time: 0.04120
Iteration: 0175 Loss: 0.63233 Time: 0.03981
Iteration: 0176 Loss: 0.63995 Time: 0.03942
Iteration: 0177 Loss: 0.62810 Time: 0.04036
Iteration: 0178 Loss: 0.63858 Time: 0.04036
Iteration: 0179 Loss: 0.63453 Time: 0.04105
Iteration: 0180 Loss: 0.63401 Ti

Iteration: 0132 Loss: 0.69595 Time: 0.04107
Iteration: 0133 Loss: 0.68939 Time: 0.04184
Iteration: 0134 Loss: 0.68698 Time: 0.04109
Iteration: 0135 Loss: 0.69047 Time: 0.04156
Iteration: 0136 Loss: 0.68463 Time: 0.04302
Iteration: 0137 Loss: 0.67737 Time: 0.04212
Iteration: 0138 Loss: 0.68337 Time: 0.04199
Iteration: 0139 Loss: 0.67796 Time: 0.04300
Iteration: 0140 Loss: 0.67841 Time: 0.04140
Iteration: 0141 Loss: 0.67891 Time: 0.04000
Iteration: 0142 Loss: 0.68029 Time: 0.04154
Iteration: 0143 Loss: 0.67126 Time: 0.04000
Iteration: 0144 Loss: 0.67238 Time: 0.04339
Iteration: 0145 Loss: 0.67379 Time: 0.04080
Iteration: 0146 Loss: 0.66731 Time: 0.04147
Iteration: 0147 Loss: 0.66535 Time: 0.04099
Iteration: 0148 Loss: 0.66722 Time: 0.04300
Iteration: 0149 Loss: 0.66678 Time: 0.04100
Iteration: 0150 Loss: 0.66742 Time: 0.04300
Iteration: 0151 Loss: 0.66318 Time: 0.04108
Iteration: 0152 Loss: 0.65922 Time: 0.04129
Iteration: 0153 Loss: 0.65879 Time: 0.04183
Iteration: 0154 Loss: 0.65956 Ti

In [5]:
# Report final results
print("FINAL RESULTS \n")

print('Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on', FLAGS.dataset)

print("All scores reported below are computed over the", FLAGS.nb_run, "run(s) \n")

print("Link prediction:\n")
print("Mean AUC score: ", np.mean(mean_roc))
print("Std of AUC scores: ", np.std(mean_roc), "\n")

print("Mean AP score: ", np.mean(mean_ap))
print("Std of AP scores: ", np.std(mean_ap), "\n \n")

FINAL RESULTS 

Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on email
All scores reported below are computed over the 10 run(s) 

Link prediction:

Mean AUC score:  0.9065097183817225
Std of AUC scores:  0.0034874404057760535 

Mean AP score:  0.9141709467315475
Std of AP scores:  0.0035821363900837115 
 

